## TensorFlow 2 Complete Project Workflow in Amazon SageMaker

> Source: [Creating a complete TensorFlow 2 workflow in Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/creating-a-complete-tensorflow-2-workflow-in-amazon-sagemaker/)
    
### Prétraitement des données -> Prototypage de code -> Réglage automatique du modèle -> Déploiement
    
1. [Introduction](#Introduction)
2. [Transformation des données avec Amazon SageMaker Processing](#SageMakerProcessing)
3. [Entrainement du modèle en mode Local](#LocalModeTraining)
4. [Point de terminaison en mode Local](#LocalModeEndpoint)
5. [Entrainement du modèle hébergé par Amazon SageMaker](#SageMakerHostedTraining)
6. [Réglage automatique du modèle](#AutomaticModelTuning)
7. [Point de terminaison hébergé par Amazon SageMaker](#SageMakerHostedEndpoint)
8. [Automatisation du flux de travail à l'aide de AWS Step Functions Data Science SDK](#WorkflowAutomation)
    1. [Ajouter une stratégie IAM à votre rôle SageMaker](#IAMPolicy)
    2. [Créer un rôle d'exécution pour Step Functions](#CreateExecutionRole)
    3. [Configurer un Pipeline d'entraînement](#TrainingPipeline)
    4. [Visualiser le flux de travail](#VisualizingWorkflow)
    5. [Créer et executer le pipeline](#CreatingExecutingPipeline)
    6. [Nettoyage](#Cleanup)
9. [Extensions](#Extensions)

    
## Introduction <a class="anchor" id="Introduction">

Si vous utilisez TensorFlow 2, vous pouvez utiliser le conteneur TensorFlow 2 préconstruit Amazon SageMaker avec des scripts d'entraînement similaires à ceux que vous utiliseriez en dehors de SageMaker. Cette fonctionnalité est nommée Mode Script. En utilisant le mode Script et d'autres fonctionnalités de SageMaker, vous pouvez créer un flux de travail complet pour un projet TensorFlow 2. Ce bloc-notes présente un tel flux de travail, y compris toutes les étapes clés telles que le prétraitement des données avec SageMaker Processing, le prototypage de code avec l'entraînement et l'inférence SageMaker Local Mode, ainsi que l'entraînement et le déploiement de modèles prêts à la production avec l'entraînement et l'inférence hébergées SageMaker. Le réglage automatique du modèle dans SageMaker permet de régler les hyperparamètres du modèle. En outre, [AWS Step Functions Data Science SDK](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/readmelink.html) est utilisé pour automatiser les principales étapes d'entraînement et de déploiement à utiliser dans un workflow de production en dehors des ordinateurs portables. 

Pour vous permettre d'exécuter ce bloc-notes dans un délai raisonnable (généralement moins d'une heure), le cas d'utilisation de ce bloc-notes est une tâche de régression simple : prédire les prix des maisons en fonction du célèbre jeu de données Boston Housing. Cet ensemble de données public contient 13 caractéristiques concernant le parc immobilier des villes de la région de Boston. Les caractéristiques comprennent le nombre moyen de chambres, l'accessibilité aux autoroutes radiales, la contiguïté de la rivière Charles, etc. 

Pour commencer, nous allons importer certains paquets nécessaires et mettre en place des répertoires pour les données d'entraînement et de test locales. Nous allons également configurer une session SageMaker pour effectuer diverses opérations et spécifier un compartiment Amazon S3 pour contenir les données d'entrée et la sortie. Le compartiment par défaut utilisé ici est créé par SageMaker s'il n'existe pas déjà et nommé conformément à l'ID de compte AWS et à la région AWS.

In [ ]:
import os, shutil
import sagemaker
import tensorflow as tf

sess = sagemaker.Session()
bucket = sess.default_bucket() 

data_dir = os.path.join(os.getcwd(), 'data')
shutil.rmtree(data_dir, ignore_errors=True)
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
shutil.rmtree(train_dir, ignore_errors=True)
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
shutil.rmtree(test_dir, ignore_errors=True)
os.makedirs(test_dir, exist_ok=True)

raw_dir = os.path.join(os.getcwd(), 'data/raw')
shutil.rmtree(raw_dir, ignore_errors=True)
os.makedirs(raw_dir, exist_ok=True)

# SageMaker Processing for dataset transformation <a class="anchor" id="SageMakerProcessing">

Ensuite, nous importerons le jeu de données et le transformerons avec SageMaker Processing, qui peut être utilisé pour traiter des téraoctets de données dans un cluster géré par SageMaker distinct de l'instance exécutant votre serveur de bloc-notes. Dans un flux de travail SageMaker typique, les ordinateurs portables ne sont utilisés que pour le prototypage et peuvent être exécutés sur des instances relativement peu coûteuses et moins puissantes, tandis que les tâches de traitement, de formation et d'hébergement de modèles sont exécutées sur des instances séparées et plus puissantes gérées par SageMaker. SageMaker Processing inclut une prise en charge standard de Scikit-Learn, ainsi qu'une option Apportez votre propre conteneur, qui peut être utilisée avec de nombreuses technologies et tâches de transformation des données. 

Tout d'abord, nous allons charger le jeu de données Boston Housing, enregistrer les données d'entités brutes et les télécharger sur Amazon S3 pour transformation par SageMaker Processing. Nous enregistrerons également les étiquettes pour la formation et les tests.

In [ ]:
import numpy as np
from tensorflow.python.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

np.save(os.path.join(raw_dir, 'x_train.npy'), x_train)
np.save(os.path.join(raw_dir, 'x_test.npy'), x_test)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)
s3_prefix = 'tf-2-workflow'
rawdata_s3_prefix = '{}/data/raw'.format(s3_prefix)
raw_s3 = sess.upload_data(path='./data/raw/', key_prefix=rawdata_s3_prefix)
print(raw_s3)

Pour utiliser SageMaker Processing, il vous suffit de fournir un script de prétraitement des données Python comme indiqué ci-dessous. Pour cet exemple, nous utilisons un conteneur Scikit-Learn préconstruit de SageMaker, qui inclut de nombreuses fonctions courantes pour le traitement des données. Il y a peu de limites sur les types de code et d'opérations que vous pouvez exécuter, et seulement un contrat minimal : les données d'entrée et de sortie doivent être placées dans des répertoires spécifiés. Si cela est fait, SageMaker Processing charge automatiquement les données d'entrée de S3 et télécharge les données transformées vers S3 une fois le travail terminé.

In [ ]:
%%writefile preprocessing.py

import glob
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

if __name__=='__main__':
    
    input_files = glob.glob('{}/*.npy'.format('/opt/ml/processing/input'))
    print('\nListe de fichiers en input: \n{}\n'.format(input_files))
    scaler = StandardScaler()
    for file in input_files:
        raw = np.load(file)
        transformed = scaler.fit_transform(raw)
        if 'train' in file:
            output_path = os.path.join('/opt/ml/processing/train', 'x_train.npy')
            np.save(output_path, transformed)
            print('Fichier avec les données d\'entrainement stocké!\n')
        else:
            output_path = os.path.join('/opt/ml/processing/test', 'x_test.npy')
            np.save(output_path, transformed)
            print('Fichier avec les données de test stocké!\n')

Avant de démarrer le travail SageMaker Processing, nous instancions un objet `SkLearnProcessor`. Cet objet vous permet de spécifier le type d'instance à utiliser dans la tâche, ainsi que le nombre d'instances. Malgré le jeu de données `Boston Housing` soit assez petit, nous utiliserons deux instances pour montrer à quel point il est facile de créer un cluster pour SageMaker Processing.

In [ ]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=get_execution_role(),
                                     instance_type='ml.m5.xlarge',
                                     instance_count=2)

Nous sommes maintenant prêts à exécuter le travail de traitement. Pour permettre la distribution égale des fichiers de données entre les instances, nous spécifions le type de distribution `ShardedByS3Key` dans l'objet `ProcessingInput`. Cela garantit que si nous avons des instances `n`, chaque instance recevra des fichiers `1/n` du compartiment S3 spécifié. Il peut prendre environ 3 minutes pour que la cellule de code suivante s'exécute, principalement pour configurer le cluster. À la fin de la tâche, le cluster sera automatiquement déchiré par SageMaker.

In [ ]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

processing_job_name = "tf-2-workflow-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination = 's3://{}/{}/data'.format(bucket, s3_prefix)

sklearn_processor.run(code='preprocessing.py',
                      job_name=processing_job_name,
                      inputs=[ProcessingInput(
                        source=raw_s3,
                        destination='/opt/ml/processing/input',
                        s3_data_distribution_type='ShardedByS3Key')],
                      outputs=[ProcessingOutput(output_name='train',
                                                destination='{}/train'.format(output_destination),
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='test',
                                                destination='{}/test'.format(output_destination),
                                                source='/opt/ml/processing/test')])

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

Dans la sortie de la tâche SageMaker Processing ci-dessus, vous devriez être en mesure de voir les journaux de deux couleurs différentes pour les deux instances différentes et que chaque instance a reçu des fichiers différents. Sans le type de distribution `ShardedByS3Key`, chaque instance aurait reçu une copie de **tous** fichiers. En répartissant les données de manière égale entre les `n` instances, vous devriez recevoir une accélération d'environ un facteur `n` pour la plupart des transformations de données sans état. Après avoir enregistré les résultats du travail localement, nous passerons au prototypage de la formation et du code d'inférence avec le mode local.

In [ ]:
train_in_s3 = '{}/train/x_train.npy'.format(output_destination)
test_in_s3 = '{}/test/x_test.npy'.format(output_destination)
!aws s3 cp {train_in_s3} ./data/train/x_train.npy
!aws s3 cp {test_in_s3} ./data/test/x_test.npy

##  Entraînement en mode Local <a class="anchor" id="LocalModeTraining">

Le mode Local d'Amazon SageMaker est un moyen pratique de s'assurer que votre code fonctionne localement comme prévu avant de passer à un entraînement hébergé à grande échelle dans un cluster distinct et plus puissant géré par Amazon SageMaker. Pour entraîner en mode Local, il est nécessaire d'avoir `docker-compose` ou `nvidia-docker-compose` (pour les instances GPU) installé. L'exécution des commandes suivantes permet d'installer `docker-compose` ou `nvidia-docker-compose` et de configurer l'environnement de bloc-notes pour vous.

In [ ]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

Ensuite, nous allons mettre en place un estimateur TensorFlow pour l'entraînement en mode local. Les paramètres clés de l'estimateur sont les suivants :

- `train_instance_type` : le type d'instance sur lequel s'exécutera la formation. Dans le cas du mode Local, nous définissons simplement ce paramètre sur `local` pour invoquer la formation en mode local sur le CPU, ou sur `local_gpu` si l'instance a un GPU (familles P3 ou G4 d'instances EC2). 
- `git_config` : pour s'assurer que les scripts d'entraînement sont centralisés pour une utilisation coordonnée et partagée par une équipe, l'estimateur peut extraire le code d'un dépôt Git plutôt que de répertoires locaux. 
- Autres paramètres à noter : les hyperparamètres de l'algorithme, qui sont transmis en tant que dictionnaire, et un paramètre booléen indiquant que nous utilisons le mode Script. 

Nous utilisons le mode Local principalement pour nous assurer que notre code fonctionne. En conséquence, au lieu d'effectuer un cycle complet d'entraînement avec de nombreuses époques (passe sur l'ensemble de données complet), nous entraînerons le modèle uniquement pour un petit nombre d'époques juste pour confirmer que le code fonctionne correctement et éviter de perdre inutilement du temps avec un entraînement à grande échelle.

In [ ]:
from sagemaker.tensorflow import TensorFlow

git_config = {'repo': 'https://github.com/aws-samples/amazon-sagemaker-script-mode', 
              'branch': 'master'}

model_dir = '/opt/ml/model'
train_instance_type = 'local'
hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01}
local_estimator = TensorFlow(git_config=git_config,
                             source_dir='tf-2-workflow/train_model',
                             entry_point='train.py',
                             model_dir=model_dir,
                             train_instance_type=train_instance_type,
                             train_instance_count=1,
                             hyperparameters=hyperparameters,
                             role=sagemaker.get_execution_role(),
                             base_job_name='tf-2-workflow',
                             framework_version='2.1',
                             py_version='py3',
                             script_mode=True)

L'appel de méthode `fit` ci-dessous démarre le travail de formation en mode local. Les mesures de formation seront enregistrées sous le code, à l'intérieur de la cellule de l'ordinateur portable. Vous devriez observer la baisse substantielle de la perte de validation au cours des cinq époques, sans erreurs d'entraînement, ce qui est une bonne indication que notre code de formation fonctionne comme prévu.

In [ ]:
%%time

inputs = {'train': f'file://{train_dir}',
          'test': f'file://{test_dir}'}

local_estimator.fit(inputs)

## Point de terminaison en Mode Local <a class="anchor" id="LocalModeEndpoint">

Bien que la formation en mode local d'Amazon SageMaker soit très utile pour s'assurer que votre code de formation fonctionne avant de passer à l'entraînement à grande échelle, il serait également utile de disposer d'un moyen pratique de tester votre modèle localement avant d'engager le temps et les dépenses nécessaires à son déploiement en production. Une possibilité est de récupérer l'artefact TensorFlow SavedModel ou un checkpoint du modèle enregistré dans Amazon S3, et de le charger dans votre bloc-notes pour les tests. Cependant, il est encore plus facile d'utiliser le SDK Python de SageMaker pour effectuer ce travail à votre place en configurant un point de terminaison en mode local.

Plus précisément, l'objet `Estimator` de la tâche d'entraînement en mode Local peut être utilisé pour déployer un modèle localement. Ce code est le même que le code que vous utiliseriez pour déployer en production. En particulier, tout ce que vous avez à faire est d'appeler la méthode de déploiement de l'estimateur local, et de la même manière que l'entraînement en mode local, spécifiez le type d'instance comme `local_gpu` ou `local` selon que votre bloc-notes se trouve sur une instance GPU ou une instance CPU. 

Juste au cas où d'autres conteneurs d'inférence s'exécutent en mode local, nous les arrêterons pour éviter les conflits avant de déployer notre nouveau modèle localement.

In [ ]:
!docker container stop $(docker container ls -aq) >/dev/null

La ligne de code suivante déploie le modèle localement dans le conteneur SageMaker TensorFlow Serving :

In [ ]:
local_predictor = local_estimator.deploy(initial_instance_count=1, instance_type='local')

Pour obtenir des prédictions à partir du point de terminaison du mode Local, appelez simplement la méthode `predict` de l'objet `Predictor`.

In [ ]:
local_results = local_predictor.predict(x_test[:10])['predictions']

Comme forme de vérification, les prédictions peuvent être comparées aux valeurs cibles réelles.

In [ ]:
local_preds_flat_list = [float('%.1f'%(item)) for sublist in local_results for item in sublist]
print('Prédictions: \t{}'.format(np.array(local_preds_flat_list)))
print('Valeurs cible: \t{}'.format(y_test[:10].round(decimals=1)))

Nous avons entraîné le modèle que pour quelques époques, et il y a beaucoup de place pour l'améliorer, mais les prévisions jusqu'à présent devraient au moins être dans un range raisonable de valeurs autour de la cible. 

Pour éviter que le conteneur SageMaker TensorFlow Serving s'exécute indéfiniment localement, fermez-le avec élégance en appelant la méthode `delete_endpoint` de l'objet `Predictor`.

In [ ]:
local_predictor.delete_endpoint()

##  Entraînement du modèle hébergé par Amazon SageMaker <a class="anchor" id="SageMakerHostedTraining">

Maintenant que nous avons confirmé que notre code fonctionne localement, nous pouvons passer à l'utilisation de la fonctionnalité d'entraînement hébergée de Amazon SageMaker. L'entraînement hébergé est préféré pour l'entraînement réel du modèle, en particulier pour un entraînement distribué à grande échelle. Contrairement à l'entraînement en mode local, pour l'entraînement hébergé, l'entraînement proprement dit n'est pas realisé sur l'instance de bloc-notes, mais sur un cluster distinct de machines gérées par SageMaker. Avant de commencer l'entraînement hébergé, les données doivent être dans S3, ou dans un système de fichiers `EFS` ou `FSx pour Lustre`. Nous allons télécharger sur S3 maintenant et confirmer que le téléchargement a réussi.

In [ ]:
s3_prefix = 'tf-2-workflow'

traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)

In [ ]:
train_s3 = sess.upload_data(path='./data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', key_prefix=testdata_s3_prefix)

inputs = {'train':train_s3, 'test': test_s3}

print(inputs)

Nous sommes maintenant prêts à configurer un objet `Estimator` pour l'entraînement hébergé. Il est similaire à l'estimateur en mode local, sauf que le `train_instance_type` a été défini sur un type d'instance SageMaker ML au lieu de `local` pour le mode local. De plus, puisque nous savons que notre code fonctionne maintenant, nous nous entraînerons pour un plus grand nombre d'époques avec l'espoir que l'entraînement des modèles convergera vers une perte de validation améliorée et plus faible.

Avec ces deux changements, nous appelons simplement `fit` pour commencer l'entraînement hébergé réel.

In [ ]:
train_instance_type = 'ml.c5.xlarge'
hyperparameters = {'epochs': 30, 'batch_size': 128, 'learning_rate': 0.01}

estimator = TensorFlow(git_config=git_config,
                       source_dir='tf-2-workflow/train_model',
                       entry_point='train.py',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-2-workflow',
                       framework_version='2.1',
                       py_version='py3',
                       script_mode=True)

Après avoir commencé le travail de formation hébergé avec l'appel de méthode `fit` ci-dessous, vous devriez observer que la formation converge sur le plus grand nombre d'époques vers une perte de validation qui est considérablement inférieure à celle obtenue dans le travail de formation en mode local plus court. On peut faire mieux ? Nous allons trouver un moyen de le faire dans la section [Réglage automatique du modèle](#AutomaticModelTuning) ci-dessous.

In [ ]:
%%time

estimator.fit(inputs)

Comme pour la formation en mode local, la formation hébergée produit un modèle enregistré dans S3 que nous pouvons récupérer. Voici un exemple de la modularité de SageMaker : après avoir formé le modèle dans SageMaker, vous pouvez maintenant sortir le modèle de SageMaker et l'exécuter n'importe où ailleurs. Vous pouvez également déployer le modèle dans un environnement prêt à la production à l'aide de la fonctionnalité de points de terminaison hébergés de SageMaker, comme indiqué dans la section [Point de terminaison avec Amazon SageMaker](#SageMakerHostedEndpoint) ci-dessous.

La récupération du modèle à partir de S3 est très facile : l'estimateur de formation hébergé que vous avez créé ci-dessus stocke une référence à l'emplacement du modèle dans S3. Vous copiez simplement le modèle de S3 en utilisant la propriété `model_data` de l'estimateur et décompressez-le pour inspecter le contenu.

In [ ]:
!aws s3 cp {estimator.model_data} ./model/model.tar.gz

L'archive décompressée doit inclure les ressources requises par TensorFlow Serving pour charger le modèle et le servir, y compris un fichier .pb :

In [ ]:
!tar -xvzf ./model/model.tar.gz -C ./model

## Réglage automatique du modèle <a class="anchor" id="AutomaticModelTuning">

Jusqu'à présent, nous avons simplement exécuté un travail d'entraînement en mode local et un travail d'entraînement hébergé sans aucune tentative réelle d'ajuster les hyperparamètres pour produire un meilleur modèle, autre que d'augmenter le nombre d'époques. Sélectionner les bonnes valeurs d'hyperparamètres pour entraîner votre modèle peut s'avérer difficile et prend généralement beaucoup de temps si vous le faites manuellement. La bonne combinaison d'hyperparamètres dépend de vos données et de votre algorithme; certains algorithmes ont de nombreux hyperparamètres différents qui peuvent être ajustés; certains sont très sensibles aux valeurs d'hyperparamètres sélectionnées; et la plupart ont une relation non linéaire entre l'ajustement du modèle et les valeurs d'hyperparamètres. SageMaker Automatic Model Tuning permet d'automatiser le processus de réglage des hyperparamètres : il exécute plusieurs tâches d'entraînement avec différentes combinaisons d'hyperparamètres pour trouver l'ensemble avec les meilleures performances du modèle.

Nous commençons par spécifier les hyperparamètres que nous voulons régler, et la plage de valeurs sur laquelle régler chacun. Nous devons également spécifier une metrique objective à optimiser : dans ce cas d'utilisation, nous aimerions minimiser la perte de validation (*validation loss*).

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
  'learning_rate': ContinuousParameter(0.001, 0.2, scaling_type="Logarithmic"),
  'epochs': IntegerParameter(10, 50),
  'batch_size': IntegerParameter(64, 256),
}

metric_definitions = [{'Name': 'loss',
                       'Regex': ' loss: ([0-9\\.]+)'},
                     {'Name': 'val_loss',
                       'Regex': ' val_loss: ([0-9\\.]+)'}]

objective_metric_name = 'val_loss'
objective_type = 'Minimize'

Ensuite, nous spécifions un objet `HyperParameterTuner` qui prend les définitions ci-dessus comme paramètres. Chaque tâche de réglage doit être doté d'un budget : un nombre maximum de tâches de formation. Une tâche de réglage se terminera après que de nombreuse tâches d'entraînement ont été exécutées. 

Nous pouvons également spécifier combien de parallélisme utiliser, dans ce cas cinq tâches, ce qui signifie que le travail de réglage sera terminé après trois séries de cinq tâches en parallèle. Pour la stratégie de réglage *Optimisation Bayesienne* par défaut utilisée ici, la recherche de réglage est éclairée par les résultats des groupes précédents de travaux de formation, donc nous n'exécutons pas tous les travaux en parallèle, mais nous divisons plutôt les travaux en groupes de travaux parallèles. Il y a un compromis : l'utilisation de tâches plus parallèles finira le réglage plus tôt, mais sacrifiera probablement la précision de recherche de réglage. 

Maintenant, nous pouvons lancer un travail de réglage d'hyperparamètres en appelant la méthode `fit` de l'objet `HyperParameterTuner`. Le travail de réglage peut prendre environ 10 minutes pour terminer. Pendant que vous attendez, l'état de la tâche de réglage, y compris les métadonnées et les résultats des tâches de formation induelles au sein de la tâche de réglage, peut être vérifié dans la console SageMaker dans le panneau *Travaux de réglage hyperparamètre*.

In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=4,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

tuning_job_name = "tf-2-workflow-{}".format(strftime("%d-%H-%M-%S", gmtime()))
tuner.fit(inputs, job_name=tuning_job_name)
tuner.wait()

Une fois le travail de réglage terminé, nous pouvons utiliser l'objet `HyperParameterTuningJobAnalytics` du SDK Python de SageMaker pour lister les 5 tâches de réglage les plus performantes. Bien que les résultats varient d'un travail de réglage à un travail de réglage, la meilleure perte de validation de la tâche de réglage (sous la colonne `FinalObjectiveValue`) sera probablement beaucoup plus faible que la perte de validation de la tâche d'entraînement hébergée ci-dessus, où nous n'avons effectué aucun réglage autre que l'augmentation manuelle du nombre des époques une fois.

In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True).head(5)

Le temps total d'entraînement et le statut des tâches de formation peuvent être vérifiés à l'aide des lignes de code suivantes. Comme l'arrêt anticipé automatique est désactivé par défaut, tous les tâches d'entraînement doivent être terminées normalement. Pour obtenir un exemple d'analyse plus approfondie d'une tâche de réglage, consultez l'exemple officiel de SageMaker [HPO_Analyze_TuningJob_Results.ipynb](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb) (en anglais).

In [ ]:
total_time = tuner_metrics.dataframe()['TrainingElapsedTimeSeconds'].sum() / 3600
print("The total training time is {:.2f} hours".format(total_time))
tuner_metrics.dataframe()['TrainingJobStatus'].value_counts()

##  Point de terminaison hebergé par Amazon SageMaker <a class="anchor" id="SageMakerHostedEndpoint">

En supposant que le meilleur modèle du travail de réglage est meilleur que le modèle produit par le travail de formation hébergé individuel ci-dessus, nous pourrions maintenant facilement déployer ce modèle en production. Une option pratique consiste à utiliser un point de terminaison hébergé par Amazon SageMaker, qui sert des prédictions en temps réel à partir du modèle entraîné (les tâches de transformation par lots (*batch processing*) sont également disponibles pour les prédictions asynchrones et hors ligne sur des jeux de données volumineux). Le point de terminaison récupère le modèle TensorFlow SavedModel créé pendant la formation et le déploie dans un conteneur SageMaker TensorFlow Serving. Tout cela peut être accompli avec une seule ligne de code. 

Plus précisément, en appelant la méthode `deploy` de l'objet `HyperParameterTuner` que nous avons instancié ci-dessus, nous pouvons déployer directement le meilleur modèle du travail de réglage vers un point de terminaison hébergé par SageMaker. Il faudra plusieurs minutes de plus pour déployer le modèle sur le point de terminaison hébergé par rapport au point de terminaison en mode local, ce qui est plus utile pour le prototypage rapide du code d'inférence.

In [ ]:
%%time

tuning_predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

Nous pouvons comparer les prédictions générées par ce point de terminaison avec celles générées localement par le point de terminaison du mode local :

In [ ]:
results = tuning_predictor.predict(x_test[:10])['predictions'] 
flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]
print('predictions: \t{}'.format(np.array(flat_list)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

Pour éviter les frais de facturation des ressources, vous pouvez supprimer le point de terminaison de prédiction pour libérer ses instances associées.

In [ ]:
sess.delete_endpoint(tuning_predictor.endpoint)

## Automatisation du flux de travail à l'aide du SDK AWS Step Functions Data Science <a class="anchor" id="WorkflowAutomation">

Dans les parties précédentes de ce bloc-notes, nous avons prototypé différentes étapes d'un projet TensorFlow 2 dans le bloc-notes lui-même. Les bloc-notes sont parfaits pour le prototypage, mais ne sont généralement pas utilisés dans les pipelines d'apprentissage automatique prêts à la production. Par exemple, un pipeline simple dans SageMaker comprend les étapes suivantes :

1. Formation du modèle.
2. Création d'un objet SageMaker Model qui enveloppe l'artefact de modèle pour l'inference.
3. Création d'une configuration de point de terminaison SageMaker spécifiant comment le modèle doit être servi (par exemple, type et quantité des instances).
4. Déploiement du modèle entraîné sur le point de terminaison SageMaker configuré. 

Le AWS Step Functions Data Science SDK automatise le processus de création et d'exécution de ce type de workflows à l'aide d'AWS Step Functions et de SageMaker. Pour faire ça, AWS Step Functions vous permet de créer des workflows à l'aide de scripts Python courts et simples qui définissent les étapes du workflow et les enchaîner. Sous le capot, toutes les étapes du flux de travail sont coordonnées par AWS Step Functions sans que vous ayez à gérer l'infrastructure sous-jacente.

Pour demarrer, installez le Step Functions Data Science SDK:  

In [ ]:
import sys

!{sys.executable} -m pip install --quiet --upgrade stepfunctions==1.1.0

### Ajouter une stratégie IAM à votre rôle SageMaker <a class="anchor" id="IAMPolicy">

**Si vous exécutez ce bloc-notes sur une instance de bloc-notes Amazon SageMaker**, le rôle IAM utilisé par votre instance de bloc-notes doit être autorisé à créer et exécuter des workflows dans AWS Step Functions. Pour accorder cette autorisation au rôle, procédez comme suit.

1. Ouvrez la Amazon [SageMaker console](https://console.aws.amazon.com/sagemaker/). 
2. Choisissez **Instance de bloc-notes** and choisissez le nom de votre instance de bloc-notes
3. Under **Autorisations et chiffrement** select the role ARN to view the role on the IAM console
4. Choisissez **Attacher des stratégies** and cherchez `AWSStepFunctionsFullAccess`.
5. Activez la case à cocher en regard de `AWSStepFunctionsFullAccess` et choisissez **Attacher la stratégie**

Si vous exécutez ce bloc-notes dans un environnement local, le SDK utilisera votre configuration de l'interface de ligne de commande AWS configurée. Pour plus d'informations, consultez [Configuration de l'AWS CLI](https://docs.aws.amazon.com/fr_fr/cli/latest/userguide/cli-chap-configure.html).


### Créer un rôle d'exécution pour Step Functions <a class="anchor" id="CreateExecutionRole">

Vous devez également créer un rôle d'exécution pour Step Functions pour permettre à ce service d'accéder à SageMaker et à d'autres fonctionnalités de service.

1. Accédez à la [console IAM](https://console.aws.amazon.com/iam/)
2. Sélectionnez **Roles**, puis **Créer un rôle**.
3. Sous **Choisir un cas d'utilisation** sélectionnez **Step Functions**
4. Choisissez **Suivant** jusqu'à ce que vous puissiez entrer un **Nom de rôle**
5. Entrez un nom tel que `StepFunctionsWorkflowExecutionRole`, puis sélectionnez**Créer un rôle**


Sélectionnez votre nouveau rôle de création et joignez-lui une stratégie. Les étapes suivantes associent une stratégie qui fournit un accès complet aux fonctions d'étape. Toutefois, à titre de bonne pratique, vous devez uniquement fournir l'accès aux ressources dont vous avez besoin. 

1. Sous l'onglet **Permissions**, cliquez sur **Ajouter une stratégie en ligne**
2. Entrez ce qui suit dans l'onglet **JSON**

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateTransformJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:StopTransformJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "sagemaker:UpdateEndpoint",
                "sagemaker:ListTags",
                "lambda:InvokeFunction",
                "sqs:SendMessage",
                "sns:Publish",
                "ecs:RunTask",
                "ecs:StopTask",
                "ecs:DescribeTasks",
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem",
                "batch:SubmitJob",
                "batch:DescribeJobs",
                "batch:TerminateJob",
                "glue:StartJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:BatchStopJobRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        }
    ]
}
```

3. Choisissez **Examiner la stratégie** et attribuez à la stratégie un nom tel que `StepFunctionsWorkflowExecutionPolicy`
4. Choisissez **Créer une stratégie**. Vous serez redirigé vers la page de détails du rôle.
5. Copiez le **ARN de rôle** en haut du **Récapitulatif**

### Configurer un Pipeline d'entraînement <a class="anchor" id="TrainingPipeline">

Bien que le SDK AWS Step Functions Data Science fournisse diverses primitives pour construire des pipelines à partir de zéro, il fournit également des modèles prédéfinis pour les workflows courants, y compris une [TrainingPipeline](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/pipelines.html#stepfunctions.Template.Pipeline.Train.TrainingPipeline) pour simplifier la création d'un pipeline de base qui inclut l'entraînement et le déploiement du modèle.

La cellule de code suivante configure un objet `pipeline` avec les paramètres nécessaires pour définir un pipeline aussi simple :

In [ ]:
import stepfunctions
from stepfunctions.template.pipeline import TrainingPipeline

# paste the StepFunctionsWorkflowExecutionRole ARN from above
workflow_execution_role = "arn:aws:iam::[YOUR_ACCOUNT_NUMBER_HERE]:role/StepFunctionsWorkflowExecutionRole"

pipeline = TrainingPipeline(
    estimator=estimator,
    role=workflow_execution_role,
    inputs=inputs,
    s3_bucket=bucket
)

### Visualiser le flux de travail <a class="anchor" id="VisualizingWorkflow">

Vous pouvez désormais afficher la définition du flux de travail et la visualiser sous forme de graphique. Ce flux de travail et ce graphique représentent votre pipeline d'entraînement, du démarrage d'une tâche d'entraînement au déploiement du modèle.

In [ ]:
print(pipeline.workflow.definition.to_json(pretty=True))

In [ ]:
pipeline.render_graph()

### Créer et executer le pipeline <a class="anchor" id="CreatingExecutingPipeline">

Avant que le workflow puisse être exécuté pour la première fois, le pipeline doit être créé à l'aide de la méthode `create` :

In [ ]:
pipeline.create()

Maintenant, le workflow peut être démarré en invoquant la méthode `execute` du pipeline :

In [ ]:
execution = pipeline.execute()

Utilisez la méthode `list_executions` pour répertorier toutes les exécutions pour le workflow que vous avez créé, y compris celle que nous venons de commencer. Une fois qu'un pipeline est créé, il peut être exécuté autant de fois que nécessaire, par exemple pour re-entraîner le modèle sur des nouvelles données. Dans le cadre ce bloc-notes, il suffit d'exécuter le workflow une fois pour économiser des ressources. La sortie inclut une liste sur laquelle vous pouvez cliquer pour accéder à une vue de l'exécution dans la console AWS Step Functions.

In [ ]:
pipeline.workflow.list_executions(html=True)

Pendant l'exécution du flux de travail, vous pouvez vérifier la progression du flux de travail dans ce bloc-notes à l'aide de la méthode `render_progress`. Cela génère un instantané de l'état actuel de votre flux de travail au fur et à mesure de son exécution. Il s'agit d'une image statique. Exécutez à nouveau la cellule pour vérifier la progression pendant l'exécution du flux de travail.

In [ ]:
execution.render_progress()

#### AVANT de procéder avec le reste du bloc-notes :

Attendez que le workflow se termine avec l'état **Succeeded**, ce qui prendra quelques minutes. Vous pouvez vérifier l'état avec `render_progress` ci-dessus, ou ouvrir dans un nouvel onglet de navigateur le lien **Inspect in AWS Step Functions** dans la sortie de cellule. 

Pour afficher les détails de l'exécution du flux de trvail terminée, depuis l'entraînement du modèle jusqu'au déploiement, utilisez la méthode `list_events`, qui répertorie tous les événements de l'exécution du flux de travail.

In [ ]:
execution.list_events(reverse_order=True, html=False)

À partir de cette liste d'événements, nous pouvons extraire le nom du point de terminaison qui a été configuré par le workflow.

In [ ]:
import re

endpoint_name_suffix = re.search('endpoint\Wtraining\Wpipeline\W([a-zA-Z0-9\W]+?)"', str(execution.list_events())).group(1)
print(endpoint_name_suffix)

Une fois que nous avons le nom du point de terminaison, nous pouvons l'utiliser pour instancier un objet `TensorFlowPredictor` qui enveloppe le point de terminaison. Ce `TensorFlowPredictor` peut être utilisé pour faire des prédictions, comme indiqué dans la cellule de code suivante. 

#### AVANT d'exécuter la cellule de code suivante :

Accédez à la [Console SageMaker](https://console.aws.amazon.com/sagemaker/), cliquez sur **Points de terminaison** dans le panneau de gauche et assurez-vous que l'état du point de terminaison soit **En service**. Si l'état est **En création**, attendez qu'il change, ce qui peut prendre plusieurs minutes.

In [ ]:
from sagemaker.tensorflow import TensorFlowPredictor

workflow_predictor = TensorFlowPredictor('training-pipeline-' + endpoint_name_suffix)

results = workflow_predictor.predict(x_test[:10])['predictions'] 
flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]
print('predictions: \t{}'.format(np.array(flat_list)))
print('target values: \t{}'.format(y_test[:10].round(decimals=1)))

Grâce au AWS Step Functions Data Science SDK, vous pouvez créer de nombreux autres flux de travail pour automatiser vos tâches de machine learning. Par exemple, vous pouvez créer un flux de travail pour automatiser l'entraînement des modèles sur une base périodique. Un tel flux de travail pourrait inclure un test de qualité du modèle après l'entraînement, avec des branches pour échouer (pas de déploiement du modèle) ou passer le test de qualité (le modèle est déployé). Parmi les autres étapes possibles, il y a le réglage automatique des modèles, le prétraitement des données avec AWS Glue, et bien plus encore. 

Pour obtenir un exemple détaillé de workflow de re-entraînement, consultez le blog AWS ML [Automating model retraining and deployment using the AWS Step Functions Data Science SDK for Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/automating-model-retraining-and-deployment-using-the-aws-step-functions-data-science-sdk-for-amazon-sagemaker/) (en anglais).

### Nettoyage <a class="anchor" id="Cleanup">

Le workflow que nous avons créé ci-dessus a déployé un modèle vers un point de terminaison. Pour éviter des frais de facturation pour un point de terminaison inutilisé, vous pouvez le supprimer à l'aide de la console SageMaker, ou en utilisant la cellule suivante. Pour faire ça, accédez à la [Console SageMaker](https://console.aws.amazon.com/sagemaker/). Cliquez ensuite sur **Points de Terminaison** dans le panneau de gauche, puis sélectionnez et supprimez tous les points de terminaison inutiles dans la liste.

In [ ]:
workflow_predictor.delete_endpoint()

## Extensions <a class="anchor" id="Extensions">

Nous avons couvert un grand nombre de contenus dans ce bloc-notes : **SageMaker Processing** pour la transformation des données, le **mode Local** pour le prototypage du code de formation et d'inférence, le **réglage automatique du modèle** et **l'entraînement et l'inférence hébergées par SageMaker**. Ce sont des éléments centraux pour la plupart des workflows de deep learning dans SageMaker. En plus, nous avons examiné comment le **AWS Step Functions Data Science SDK** aide à automatiser les flux de travail de deep learning après avoir terminé la phase de prototypage d'un projet.

Outre toutes les fonctionnalités de SageMaker décrites ci-dessus, de nombreuses autres fonctionnalités peuvent être appliqués à votre projet. Par exemple, pour gérer les problèmes pendant la phase d'entraînement du modèle de Deep Learning, tels que la disparition ou l'explosion de gradients, **SageMaker Debugger** est utile. Pour gérer les problèmes courants tels que la dérive des données après la production d'un modèle, **SageMaker Model Monitor peut être appliqué.